Configurar TASK e baixar

In [2]:
!wget http://data.allenai.org.s3.amazonaws.com/downloads/SciTailV1.1.zip
!unzip SciTailV1.1.zip

TASK = 'scitail_1.0'

splits = [
 f'SciTailV1.1/tsv_format/{TASK}_train.tsv', 
 f'SciTailV1.1/tsv_format/{TASK}_test.tsv',
 f'SciTailV1.1/tsv_format/{TASK}_dev.tsv',
]

label_idx = [
             0,
             1,
            ]

--2020-08-11 21:12:32--  http://data.allenai.org.s3.amazonaws.com/downloads/SciTailV1.1.zip
Resolving data.allenai.org.s3.amazonaws.com (data.allenai.org.s3.amazonaws.com)... 52.218.250.19
Connecting to data.allenai.org.s3.amazonaws.com (data.allenai.org.s3.amazonaws.com)|52.218.250.19|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14174621 (14M) [application/zip]
Saving to: ‘SciTailV1.1.zip’

SciTailV1.1.zip     100%[===================>]  13.52M  58.9MB/s    in 0.2s    

2020-08-11 21:12:32 (58.9 MB/s) - ‘SciTailV1.1.zip’ saved [14174621/14174621]



Prepare input

In [8]:
!pip install transformers --upgrade
!pip install mosestokenizer

# Translate sentences to English and save mapping dicitionary as json.
# Based on https://github.com/ruanchaves/assin/blob/master/sources/translate.py

import json
import math
import os
import torch

from tqdm.notebook import tqdm
from transformers import MarianMTModel, MarianTokenizer

model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to("cuda")

def decode(text):
    decoded = tokenizer.decode(
                    text, 
                    skip_special_tokens=True,
                    )

    return decoded

def needs_translation(sample, translations):
    needs = [text for text in sample if text not in translations.keys()]
    
    return needs

def translation(sample):
    src_txts = [f'>>pt_BR<< {text}' for text in sample]

    model_inputs = tokenizer.prepare_translation_batch(src_txts).to("cuda")
    generated_ids = model.generate(
                                    model_inputs.input_ids, 
                                    attention_mask=model_inputs.attention_mask, 
                                    num_beams=2
                                    )
    
    translated = [decode(text) for text in generated_ids]

    return translated


def translate2dict(sentences, dictpath, batch_size):
    if not os.path.isfile(dictpath):
        with open(dictpath, 'w') as f:
            json.dump({}, f)

    with open(dictpath) as f:
        translations = json.load(f)
    
    remaining = needs_translation(sentences, translations)
    batch = math.ceil(len(remaining)/batch_size)

    for idx in tqdm(range(batch), "Translating"):
        keys = remaining[idx*batch_size:(idx+1)*batch_size] 
        values = translation(keys)
    
        new_translations = dict(zip(keys, values))
        translations.update(new_translations)
        torch.cuda.empty_cache()
        with open(dictpath, 'w+') as f:
            json.dump(translations, f)

from pprint import pprint
import pandas as pd
import os

sentences = list()
length = 0

for f in splits:
    table = pd.read_csv(f, sep='\t', error_bad_lines=False, header=None)
    print(f) 
    #print(table.head())
    #print('\n'*3)
    
    for idx in label_idx:
        label = table[idx].copy()
        print(label)
        print('\n'*2)

        sentences.extend(list(label))
        length = length + label.size
        


assert length == len(sentences)
sentences = set(sentences)

Requirement already up-to-date: transformers in /usr/local/lib/python3.6/dist-packages (3.0.2)
SciTailV1.1/tsv_format/scitail_1.0_train.tsv
0        Pluto rotates once on its axis every 6.39 Eart...
1        ---Glenn =====================================...
2        geysers - periodic gush of hot water at the su...
3        Facts: Liquid water droplets can be changed in...
4        By comparison, the earth rotates on its axis o...
                               ...                        
23092    which is not only the motion of our bodies, bu...
23093    The Red Star, that celestial curse whose eccen...
23094    The lines in the spectrum of a luminous body s...
23095    The radial velocity of a star away from or tow...
23096    This expansion causes the light from distant s...
Name: 0, Length: 23097, dtype: object



0         Earth rotates on its axis once times in one day.
1         Earth rotates on its axis once times in one day.
2        The surface of the sun is much hotter than 

Translate

In [9]:
dictpath = f'{TASK}.json'
translate2dict(list(sentences),  dictpath, 100)

Replace with translation

In [12]:
from os import makedirs
import json

with open(dictpath) as reader:
    dictionary = json.load(reader)

translation_folder = 'translation/SciTailV1.1/tsv_format/'
makedirs(translation_folder)
for f in splits:
    table = pd.read_csv(f, sep='\t', header=None, error_bad_lines=False)
    for idx in label_idx:
        table[idx] = table[idx].map(dictionary)
    table.to_csv(f'translation/{f}', index=None, header=None, sep='\t')